## **INSTALACIÓN Y CARGA DE PAQUETES**

In [ ]:
!pip install  numpy
!pip install  pandas 
!pip install openpyxl
!pip install matplotlib

In [ ]:
import numpy as np
import pandas as pd
import openpyxl
import matplotlib.pyplot as plt
import datetime
from scipy.interpolate import make_interp_spline
from datetime import datetime


from openpyxl import load_workbook
from matplotlib.ticker import (MultipleLocator, FormatStrFormatter, AutoMinorLocator)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## **INGRESOS**

In [ ]:
def lee_archivo(nombre):
  path='/content/drive/MyDrive/Fundacion Frontera Economica/Muni/python/inputs'
  filename = path + "/" + nombre.upper() + '.xlsx'

  input_cols=[0,2,5,8] # Columnas a importar
  df = pd.read_excel(filename,
                   header=0,
                   usecols = input_cols, 
                   index_col=False,
                   )
  
  pd.options.display.float_format = '${:0,.0f}'.format
  pd.set_option('display.float_format', lambda x: '%.2f' % x) # elimino notación cientifica


  df.columns = ['n_tasa','c_tasa','ing_mensual','concepto']

  concepto = pd.read_excel('/content/drive/MyDrive/Fundacion Frontera Economica/Muni/python/inputs/Concepto.xlsx')
  df = df.merge(concepto, how='right', on='n_tasa')

  df = df.groupby(by='CONCEPTO', as_index=False).sum() 

  df = df.transpose().drop(['n_tasa'], axis=0)

  df['Periodo'] = nombre
  df = df.set_index('Periodo')
  
  columnas = ['-', 'COMERCIO', 'COPARTICIPACION', 'INGRESOS INDIRECTOS', 'INMUEBLE', 'IPA', 'OTRAS TASAS', 'OTROS INGRESOS']
  df.columns = columnas
  df = df.iloc[1:]

  df.head()
  return df

In [ ]:
def tabla2020(meses2020:list):
    ingresos2020 = pd.concat([lee_archivo(f) for f in meses2020])
    ingresos2020 = ingresos2020.drop(['-'],axis=1)
    ingresos2020["TOTAL"] = ingresos2020.sum(axis=1)

    return ingresos2020

def tabla2021(meses2021:list):
    ingresos2021 = pd.concat([lee_archivo(f) for f in meses2021])
    ingresos2021 = ingresos2021.drop(['-'],axis=1)
    ingresos2021["TOTAL"] = ingresos2021.sum(axis=1)
    
    return ingresos2021

def compilado(ing2020, ing2021):
    ingresos = pd.concat([ing2020, ing2021])
    ingresos["TOTAL"] = ingresos.sum(axis=1)

    return ingresos

In [ ]:
meses2020 = ['ENE-2020','FEB-2020','MAR-2020','ABR-2020','MAY-2020','JUN-2020','JUL-2020','AGO-2020','SEP-2020','OCT-2020','NOV-2020','DIC-2020']
meses2021 = ['ENE-2021','FEB-2021','MAR-2021','ABR-2021']
ingresos = compilado(tabla2020(meses2020),tabla2021(meses2021))
ingresos.head(20)

,COMERCIO,COPARTICIPACION,INGRESOS INDIRECTOS,INMUEBLE,IPA,OTRAS TASAS,OTROS INGRESOS,TOTAL
Periodo,,,,,,,,
ENE-2020,140263772.73,157779505.84,434123.00,35012908.38,60377320.63,17552793.11,6711919.72,836264686.82
FEB-2020,125045144.14,142945183.85,211394.00,28677254.23,22601077.74,19758353.12,6419258.88,691315331.92
MAR-2020,97323022.04,158394305.12,134824.00,12114857.35,20729859.96,16034515.91,5775660.65,621014090.06
ABR-2020,120227434.64,116522984.92,29501.00,4979773.51,7931010.36,5478086.49,1117257.07,512572095.98
MAY-2020,124546276.77,126291534.19,56461.00,8792851.47,16137441.89,14134529.39,3248575.87,586415341.16
JUN-2020,134911061.63,204273380.76,82046.00,15093031.06,18525873.37,15731309.06,4709808.04,786653019.84
JUL-2020,137264315.66,177194046.14,83026.00,17709610.17,23802693.56,16667967.92,5636605.25,756716529.40
AGO-2020,157453939.21,218332017.34,226146.00,15917504.94,12954031.68,17048294.07,6448034.35,856759935.18
SEP-2020,150351402.70,204993765.34,215613.00,17310397.40,16925694.90,19762322.40,8311454.90,835741301.28


## **BOCA DE PAGO**

In [ ]:
def bocapago(nombre):
  path='/content/drive/MyDrive/Fundacion Frontera Economica/Muni/BOCAS DE PAGO/Base Historico'
  filename = path + "/" + nombre.upper() + '.xlsx'

  input_cols=[0,1,2,3,4] # Columnas a importar
  df = pd.read_excel(filename,
                   header=0,
                   usecols = input_cols, 
                   index_col=False,
                   )
  
  df.columns = ['n_tasa','Fecha','Lugar','Importe', 'CONCEPTO']
  
  df = df.drop(columns=['Fecha','n_tasa'])

  df = pd.pivot_table(df, values='Importe', index='CONCEPTO', columns='Lugar', aggfunc='sum').fillna(0)
  df = df.assign(Total=df.sum(1))
  df = df.rename(columns={'Total':'TOTAL GENERAL'})
  df = df.rename(index={'Ingresos Directos':'INGRESOS DIRECTOS', 'Inmueble':'INMUEBLE','Comercio':'COMERCIO','Otras Tasas':'OTRAS TASAS','Otros Ingresos':'OTROS INGRESOS'})

  columnas = ['TESORERIA', 'BANCOS', 'IPLYC', 'RAPIPAGOS', 'PAGO FACIL', 'PAGO MIS CUENTAS', 'RED LINK', 'VISA', 'NARANJA', 'MACRO ONLINE', 'PAGOS 360', 'TOTAL GENERAL']
  df.columns = columnas

  df.head()
  return df

In [ ]:
def IACM(nombre):
  path='/content/drive/MyDrive/Fundacion Frontera Economica/Muni/python/inputs'
  filename = path + "/" + nombre.upper() + '.xlsx'

  input_cols=[0,2,5,8] # Columnas a importar
  # Importo los datos
  df = pd.read_excel(filename,
                   header=0,
                   usecols = input_cols, 
                   index_col=False,
                   )
  
  pd.options.display.float_format = '${:0,.0f}'.format
  pd.set_option('display.float_format', lambda x: '%.2f' % x) # elimino notación cientifica

  df.columns = ['n_tasa','c_tasa','ing_mensual','concepto']

  concepto = pd.read_excel('/content/drive/MyDrive/Fundacion Frontera Economica/Muni/python/inputs/Concepto.xlsx')
  df = df.merge(concepto, how='right', on='n_tasa')

  df = df.groupby(by='CONCEPTO', as_index=False).sum() 
  df = df.drop(columns='n_tasa')
  df = df.rename(columns={'ing_mensual':'TOTAL IACM'})
  df = df.fillna(0)

  df.head()
  return df

In [ ]:
df = bocapago('01ENERO2021').merge(IACM('ENE-2021'), how='left', on='CONCEPTO').fillna(0)
df = df.set_index('CONCEPTO')
df['DIFERENCIA'] = df['TOTAL IACM'] - df['TOTAL GENERAL']
df.head()

,TESORERIA,BANCOS,IPLYC,RAPIPAGOS,PAGO FACIL,PAGO MIS CUENTAS,RED LINK,VISA,NARANJA,MACRO ONLINE,PAGOS 360,TOTAL GENERAL,TOTAL IACM,DIFERENCIA
CONCEPTO,,,,,,,,,,,,,,
COMERCIO,42581763.65,10800.00,249182.20,5530067.58,6966316.14,0.00,0.00,0.00,0.00,198815.89,13770895.88,69307841.34,204355151.73,135047310.39
INGRESOS DIRECTOS,2902112.88,0.00,28235.40,371638.60,162146.20,3424.00,874.00,15523.80,4521.00,253437.60,367932.60,4109846.08,0.00,-4109846.08
INMUEBLE,25077716.30,0.00,609573.68,5862694.27,3434894.65,493504.00,132024.87,224634.60,78783.00,2804158.68,4383004.00,43100988.05,44640874.61,1539886.56
OTRAS TASAS,12209540.52,3950475.38,40895.96,398672.15,434691.27,0.00,0.00,0.00,0.00,36126.26,38587.84,17108989.38,24518006.05,7409016.67
OTROS INGRESOS,4340245.15,267075.15,46424.76,537445.50,317727.26,309.65,207.74,0.00,0.00,629417.84,1018943.68,7157796.73,8075430.05,917633.32
